In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_filmes = pd.read_csv('data/movies.csv')
df_notas = pd.read_csv('data/ratings.csv')

In [5]:
df_filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#Usando Regex eu consigo separar o ano do titulo do filme
df_filmes['year'] = df_filmes.title.str.extract('(\(\d\d\d\d\))',expand=False)
#remover os parenteses
df_filmes['year'] = df_filmes.year.str.extract('(\d\d\d\d)',expand=False)
#Removendo o ano do filme
df_filmes['title'] = df_filmes.title.str.replace('(\(\d\d\d\d\))', '')
#Aplicando a função strip para remover qualquer caracter em branco
df_filmes['title'] = df_filmes['title'].apply(lambda x: x.strip())
df_filmes.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Vamos remover o genero, pois ele não é necessário nesse tipo de sistema de recomendação

In [8]:
df_filmes = df_filmes.drop('genres',1)

In [9]:
df_filmes.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [10]:
df_notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
df_notas = df_notas.drop('timestamp',1)

In [14]:
df_notas.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


## Filtragem colaborativa

É conhecida como filtragem Usuário-Usuário. Como o proprio nome diz, esta tecnica usa outros usuários para recomendar itens ao usuário de entrada.
Tenta encontrar usuário com gostos parecidos como entrada e então recomenda itens que você possa gostar.

Há varios métodos para encontrar a similaridade( mesmo alguns usando técnicas de ML) e oque foi ensinado é baseado na função de Correlação de Pearson.

O processo para criar um sistema de recomendação baseado no usuário é o seguinte:
    1. Selecione um usuário com os filmes que o usuário assistiu.
    2. Baseado em sua classificação para filmes, encontre os principais vizinhos X.
    3. Obter o registro do filme assistido do usuário para cada vizinho.
    4. Calcular a pontuação de similaridade usando a formula.
    5. Recomende os itens com a maior pontuação.

In [15]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


In [18]:
#Filtrar os filmes pelo titulo
inputId = df_filmes[df_filmes['title'].isin(inputMovies['title'].tolist())]
inputId

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
257,296,Pulp Fiction,1994
973,1274,Akira,1988
1445,1968,"Breakfast Club, The",1985


In [20]:
#Então mesclá-lo para que possamos obter o movieId. Está implicitamente mesclando-o pelo título.
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


In [21]:
#Retirar oque não preciso
inputMovies = inputMovies.drop('year', 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [22]:
#Filtrando usuários que assistiram a filmes que a entrada assistiu e armazenando
userSubset = df_notas[df_notas['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [28]:
#Groupby cria vários subframes de dados em que todos eles têm o mesmo valor na coluna especificada como o parâmetro
userSubsetGroup = userSubset.groupby(['userId'])


In [42]:
#Posso filtrar pela ID do usuário, neste caso eu to pegando todos os filmes do usuário com a ID 15
userSubsetGroup.get_group(608)

,userId,movieId,rating
98666,608,1,2.5
98667,608,2,2.0
98723,608,296,5.0
98882,608,1274,4.0
98980,608,1968,4.0


In [39]:
userSubsetGroup.groups

{1: Int64Index([0, 16], dtype='int64'),
 4: Int64Index([320, 422], dtype='int64'),
 5: Int64Index([516, 533], dtype='int64'),
 6: Int64Index([560, 692], dtype='int64'),
 7: Int64Index([874], dtype='int64'),
 8: Int64Index([1026, 1049], dtype='int64'),
 10: Int64Index([1119], dtype='int64'),
 14: Int64Index([1403], dtype='int64'),
 15: Int64Index([1434, 1442], dtype='int64'),
 16: Int64Index([1575], dtype='int64'),
 17: Int64Index([1667, 1676], dtype='int64'),
 18: Int64Index([1772, 1773, 1796], dtype='int64'),
 19: Int64Index([2274, 2275, 2629], dtype='int64'),
 20: Int64Index([2977], dtype='int64'),
 21: Int64Index([3219, 3220, 3231], dtype='int64'),
 23: Int64Index([3789, 3835], dtype='int64'),
 24: Int64Index([3907], dtype='int64'),
 26: Int64Index([4048], dtype='int64'),
 27: Int64Index([4059, 4060], dtype='int64'),
 28: Int64Index([4215], dtype='int64'),
 29: Int64Index([4768], dtype='int64'),
 31: Int64Index([4879], dtype='int64'),
 32: Int64Index([4929, 4961], dtype='int64'),
 3

In [43]:
# Classificando para que os usuários com filme mais em comum com a entrada tenham prioridade
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [44]:
userSubsetGroup[0:3]

[(91,        userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0), (177,        userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5), (219,        userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0)]

## Similaridade dos usuários para o usuário de entrada

Em seguida, vamos comparar todos os usuários (não todos !!!) ao nosso usuário especificado e encontrar aquele que é mais semelhante.
Vamos descobrir como cada usuário é semelhante à entrada por meio do coeficiente de correlação de Pearson, que é usado para medir a força de uma associação linear entre duas variáveis.



### Por que a correlação de Pearson?

A correlação de Pearson é invariante ao escalonamento, isto é, multiplicando todos os elementos por uma constante diferente de zero ou adicionando qualquer constante a todos os elementos. 

Por exemplo, se você tem dois vetores X e Y, então, pearson (X, Y) == pearson (X, 2 * Y + 3). 

Esta é uma propriedade muito importante nos sistemas de recomendação porque, por exemplo, dois usuários podem classificar duas séries de itens totalmente diferentes em termos de taxas absolutas, mas seriam usuários semelhantes (ou seja, com idéias semelhantes) com taxas semelhantes em várias escalas.

Os valores fornecidos pela fórmula variam de r = -1 a r = 1, onde 1 forma uma correlação direta entre as duas entidades (significa uma correlação positiva perfeita) e -1 forma uma correlação negativa perfeita.

No nosso caso, um 1 significa que os dois usuários têm gostos semelhantes, enquanto -1 significa o oposto.

In [45]:
# Posso impor um limite por onde iremos percorrer, para que não precisaremos passar por todos
#userSubsetGroup = userSubsetGroup[0:100]

In [52]:
# Armazene a Correlação de Pearson em um dicionário, em que a chave é o ID do usuário e o valor é o coeficiente
correlacaoPearson = {}
#Percorrer os usuários no dataset
for name, group in userSubsetGroup:
    # Vamos começar classificando a entrada e o grupo de usuários atual para que os valores não sejam misturados mais tarde
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Pegando o N da formula
    nNotas = len(group)
    #Pegando os filmes que tem em comum
    temp_df = inputMovies[inputMovies['movieId'].isin(group.movieId.tolist())]
    temp_notas = temp_df['rating'].tolist()
    temp_group = group['rating'].tolist()
    #Agora vamos calcular a Correlação de Pearson entre 2 usuários, chamados X e Y
    Sxx = sum([i**2 for i in temp_notas]) - pow(sum(temp_notas),2)/float(nNotas)
    Syy = sum([i**2 for i in temp_group]) - pow(sum(temp_group),2)/float(nNotas)
    Sxy = sum( i*j for i, j in zip(temp_notas, temp_group)) - sum(temp_notas)*sum(temp_group)/float(nNotas)
    #Se o denominador for diferente de 0 então divida.
    if Sxx != 0 and Syy != 0:
        correlacaoPearson[name] = Sxy/sqrt(Sxx*Syy)
    else:
        correlacaoPearson[name] = 0
    

In [54]:
correlacaoPearson.items()

dict_items([(91, 0.43852900965351443), (177, 0.0), (219, 0.45124262819713973), (274, 0.716114874039432), (298, 0.9592712306918567), (414, 0.9376144618769914), (474, 0.11720180773462392), (477, 0.4385290096535153), (480, 0.7844645405527362), (483, 0.08006407690254357), (599, 0.7666866491579839), (608, 0.920736884379251), (50, 0.15713484026367722), (57, -0.7385489458759964), (68, 0.0), (103, 0.5222329678670935), (135, 0.8703882797784892), (182, 0.9428090415820635), (202, 0.5222329678670935), (217, 0.30151134457776363), (226, 0.9438798074485389), (288, 0.6005325641789633), (307, 0.9655810287305759), (318, 0.44486512077567225), (322, 0.5057805388588731), (330, 0.9035942578600878), (357, 0.5606119105813882), (434, 0.9864036607532465), (448, 0.30151134457776363), (469, 0.8164965809277261), (561, 0.5222329678670935), (600, 0.18442777839082938), (606, 0.9146591207600472), (610, -0.47140452079103173), (18, 1.0), (19, -0.5), (21, 0), (45, 0.5000000000000009), (63, -0.4999999999999982), (64, 0.0)

In [56]:
pearsonDF = pd.DataFrame.from_dict(correlacaoPearson, orient='index')
pearsonDF.columns = ['Similaridade']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,Similaridade,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [58]:
topUsers=pearsonDF.sort_values(by='Similaridade', ascending=False)[0:50]
topUsers.head()

,Similaridade,userId
43,1.0,132
181,1.0,382
219,1.0,602
130,1.0,130
129,1.0,125


In [59]:
#Agora vamos multiplicar pelo peso das notas.
topUsersRating=topUsers.merge(df_notas, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,Similaridade,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


In [60]:
topUsersRating['peso'] = topUsersRating['Similaridade']*topUsersRating['rating']
topUsersRating.head()

,Similaridade,userId,movieId,rating,peso
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


In [61]:
#Aplica uma soma aos principais usuários após agrupá-lo por userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['Similaridade','peso']]
tempTopUsersRating.columns = ['sum_similaridade','sum_peso']
tempTopUsersRating.head()

,sum_similaridade,sum_peso
movieId,,
1,36.0,124.0
2,18.0,58.0
3,3.0,11.0
5,3.0,8.5
6,13.0,49.5


In [63]:
#Criar dataframe novo
recommendation_df = pd.DataFrame()
#Agora pegamos a média dos pesos
recommendation_df['Média do peso da similaridade'] = tempTopUsersRating['sum_peso']/tempTopUsersRating['sum_similaridade']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,Média do peso da similaridade,movieId
movieId,,
1,3.444444,1
2,3.222222,2
3,3.666667,3
5,2.833333,5
6,3.807692,6


In [64]:
recommendation_df = recommendation_df.sort_values(by='Média do peso da similaridade', ascending=False)
recommendation_df.head(10)

,Média do peso da similaridade,movieId
movieId,,
3035,5.0,3035
1956,5.0,1956
2522,5.0,2522
2495,5.0,2495
2477,5.0,2477
163386,5.0,163386
2455,5.0,2455
2450,5.0,2450
2427,5.0,2427


In [67]:
df_filmes.loc[df_filmes['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
1433,1956,Ordinary People,1980
1826,2427,"Thin Red Line, The",1998
1842,2450,Howard the Duck,1986
1846,2455,"Fly, The",1986
1865,2477,Firewalker,1986
1877,2495,"Fantastic Planet, The (Planète sauvage, La)",1973
1898,2522,Airport '77,1977
2288,3035,Mister Roberts,1955
9380,163112,Winnie the Pooh Goes Visiting,1971
9382,163386,Winnie the Pooh and the Day of Concern,1972


'[alt' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.
